[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20DistilBertForTokenClassification.ipynb)

## Import DistilBertForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.2.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for token classification via `BertForTokenClassification` or `TFBertForTokenClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDistilBertForTokenClassification](https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertfortokenclassification)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.8.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.8.1 tensorflow==2.4.1

     |████████████████████████████████| 2.5 MB 7.9 MB/s 
     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 2.9 MB 43.6 MB/s 
     |████████████████████████████████| 462 kB 79.4 MB/s 
     |████████████████████████████████| 3.8 MB 37.6 MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [elastic/distilbert-base-cased-finetuned-conll03-english](elastic/distilbert-base-cased-finetuned-conll03-english) model from HuggingFace as an example
- In addition to `TFDistilBertForTokenClassification` we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import TFDistilBertForTokenClassification, DistilBertTokenizer 

MODEL_NAME = 'elastic/distilbert-base-cased-finetuned-conll03-english'

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# let's add from_pt=True since there is no TF weights available for this model
# from_pt=True will convert the pytorch model to tf model

model = TFDistilBertForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)
model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

All PyTorch model weights were used when initializing TFDistilBertForTokenClassification.

All the weights of TFDistilBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForTokenClassification for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./elastic/distilbert-base-cased-finetuned-conll03-english/saved_model/1/assets


INFO:tensorflow:Assets written to: ./elastic/distilbert-base-cased-finetuned-conll03-english/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 254812
-rw-r--r-- 1 root root       959 Aug  9 09:22 config.json
drwxr-xr-x 3 root root      4096 Aug  9 09:22 saved_model
-rw-r--r-- 1 root root 260917856 Aug  9 09:22 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 4268
drwxr-xr-x 2 root root    4096 Aug  9 09:22 assets
-rw-r--r-- 1 root root 4361486 Aug  9 09:22 saved_model.pb
drwxr-xr-x 2 root root    4096 Aug  9 09:22 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 220
-rw-r--r-- 1 root root    112 Aug  9 09:21 special_tokens_map.json
-rw-r--r-- 1 root root    603 Aug  9 09:21 tokenizer_config.json
-rw-r--r-- 1 root root 213450 Aug  9 09:21 vocab.txt


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.txt` from the tokenizer
- All we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [6]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}

In [7]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [8]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 216
-rw-r--r-- 1 root root     51 Aug  9 09:22 labels.txt
-rw-r--r-- 1 root root 213450 Aug  9 09:22 vocab.txt


## Import and Save DistilBertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DistilBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` only accepts local paths and not distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. That is why we use `write.save` so we can use `.load()` from any file systems



In [10]:
from sparknlp.annotator import *

tokenClassifier = DistilBertForTokenClassification.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DistilBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [12]:
! ls -l {MODEL_NAME}_spark_nlp

total 236096
-rw-r--r-- 1 root root 241751684 Aug  9 09:30 distilbert_classification_tensorflow
drwxr-xr-x 5 root root      4096 Aug  9 09:28 fields
drwxr-xr-x 2 root root      4096 Aug  9 09:28 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DistilBertForTokenClassification model 😊 

In [13]:
tokenClassifier_loaded = DistilBertForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("ner")

In [14]:
tokenClassifier_loaded.getCaseSensitive()

True

That's it! You can now go wild and use hundreds of `DistilBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀 
